In [6]:
from dotenv import load_dotenv

load_dotenv()

from openai import OpenAI

openai = OpenAI()


In [24]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
#from neo4jconnect import Neo4jConnection

from aiohttp import ClientSession
from bs4 import BeautifulSoup


app = FastAPI(title="Nonprofit Recommendaation Agent", description="Search and summarize nonprofits and recommend nonprofits personalized to user prompts")

#neo4j_conn = Neo4jConnection(uri="neo4j://neo4j:7687", user="neo4j", password="secretpassword")

async def scrape(query: str) -> list: #doesn't work 
    async with ClientSession() as session:
        url = f"https://www.duckduckgo.com/search?q={query.replace(' ', '+')}+site:.org" #can remove the .org
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            links = []
            for link in soup.select('a'):
                href = link.get('href')
                if href and "url?q=" in href:  # Filter to find the actual URL
                    url = href.split("url?q=")[1].split("&")[0]
                    links.append(url)
            return links[:5]
        
async def generate_search_keywords(prompt: str) -> List[dict]:
    



In [7]:
import httpx
import openai
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import pandas as pd

SERP_API_KEY = "1c92173370dc002dd7a0053eb47eeef87f9cce81bc61ccdcc766c1672a4b0087"

SEARCH_ASSISTANT = """
You are a non-profit expert and people ask you for help looking for the best educational non-profit for their needs.
Your job is to create a search query based on a user's prompt, that will find non-profit orginizations when
typed into a search engine.
"""

async def search_serpapi(prompt: str) -> list:
    print(f"searchapi function {prompt}")
    query = prompt
    url = f"https://serpapi.com/search.json?q={query}&engine=google&api_key={SERP_API_KEY}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        data = response.json()
        
        # Extract URLs and snippets from organic results
        results = [
            {
                "title": item["title"],
                "link": item["link"],
                "snippet": item.get("snippet", "")
            }
            for item in data.get("organic_results", [])[:5]
        ]
        
        return results

async def summarize_with_ai(text: str, url: str) -> str:
    response = openai.chat.completions.create(
        model='anthropic.claude-3.5-sonnet.v2',
        messages=[
            {
                'role': 'user',
                'content': f"Summarize this snippet about a non-profit:\n\n{text}\n\nInclude the context from this website: {url}"
            },
        ],
    )
    # Extract the content from the response
    summary = response.choices[0].message.content
    return summary.strip()

async def fetch_full_page(url: str) -> str:
    async with ClientSession() as session:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            paragraphs = soup.find_all('p')
            text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 50])
            return text[:3000].strip()
        
# Fetch top 5 summaries of websites similar to the prompt
async def full_search_summarize(prompt: str) -> list:
    print(f"Starting SerpAPI search for prompt: {prompt}")
    
    search_results = await search_serpapi(prompt)
    print(f"Ending SerpAPI search for prompt: {prompt}")
    print(search_results)

    summaries = []
    for result in search_results:
        # Await the coroutine so that we get the actual summary text
        snippet_summary = await summarize_with_ai(result["snippet"], result["link"])
        summaries.append({
            "title": result["title"],
            "link": result["link"],
            "summary": snippet_summary
        })
    return summaries

user_prompt_ex = "financial aid educational non profit"

async def generate_table_from_prompt(prompt: str):
    # Perform the full search and summarization
    search_results = await full_search_summarize(prompt)
    
    # Prepare data for the table
    data = [{"Link": result["link"], "Summary": result["summary"]} for result in search_results]
    
    # Create a DataFrame for better visualization
    df = pd.DataFrame(data)
    
    # Print the table
    print(df.to_string(index=False))

# Run the async function (this line assumes you're in an async-capable environment)
await generate_table_from_prompt(user_prompt_ex)


Starting SerpAPI search for prompt: financial aid educational non profit
searchapi function financial aid educational non profit
Ending SerpAPI search for prompt: financial aid educational non profit
[{'title': 'NASFAA.org Home', 'link': 'https://www.nasfaa.org/', 'snippet': 'NASFAA is a nonpartisan, nonprofit association that provides professional development and services for financial aid administrators.'}, {'title': 'Types of Financial Aid: Loans, Grants, and Work-Study ...', 'link': 'https://studentaid.gov/understand-aid/types', 'snippet': 'Financial aid is money to help pay for college or career school. Grants, work-study, loans, and scholarships help make college or career school affordable.'}, {'title': 'Working in Support of Education', 'link': 'https://www.wise-ny.org/', 'snippet': 'W!se is an award-winning national educational non-profit dedicated to improving economic mobility through programs that develop financial literacy and readiness ...'}, {'title': 'Find Aid You Need 

In [14]:
import httpx
import openai
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import pandas as pd
import asyncio

SERP_API_KEY = "1c92173370dc002dd7a0053eb47eeef87f9cce81bc61ccdcc766c1672a4b0087"

SEARCH_ASSISTANT = """
You are a non-profit expert and people ask you for help looking for the best educational non-profit for their needs.
Your job is to create a search query based on a user's prompt, that will find non-profit organizations when
typed into a search engine.
"""

async def search_serpapi(prompt: str) -> list:
    """
    Perform a single SERP API search request for the given prompt.
    """
    query = prompt
    url = f"https://serpapi.com/search.json?q={query}&engine=google&api_key={SERP_API_KEY}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        data = response.json()
        results = [
            {
                "title": item.get("title", ""),
                "link": item.get("link", ""),
                "snippet": item.get("snippet", "")
            }
            for item in data.get("organic_results", [])
        ]
    return results

async def fetch_full_page(url: str) -> str:
    """
    Retrieve and return a limited amount of text from the webpage at the given URL.
    """
    async with ClientSession() as session:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            paragraphs = soup.find_all('p')
            text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 50])
            return text[:3000].strip()

async def is_nonprofit(text: str, url: str) -> bool:
    """
    Determine—via an AI call—whether the given snippet (or full page if snippet is empty)
    suggests that the website represents a nonprofit organization.
    The AI is instructed to respond with a simple 'Yes' or 'No'.
    """
    # If no snippet is available, fetch additional content.
    if not text:
        text = await fetch_full_page(url)
    
    response = openai.chat.completions.create(
        model='anthropic.claude-3.5-sonnet.v2',
        messages=[
            {
                'role': 'user',
                'content': (
                    f"Based on the following snippet and URL, determine if the website likely represents a nonprofit organization. "
                    f"Do not attempt to read from the url website. Instead just make a guess from the webpage name"
                    f"You must be VERY confident"
                    f"Answer only 'Yes' or 'No'.\n\nSnippet: {text}\n\nURL: {url}"
                )
            },
        ],
    )
    answer = response.choices[0].message.content.strip().lower()
    if not (answer.startswith("yes") or answer.startswith("no")):
        print("SCRAPING")
        additional_text = await fetch_full_page(url)
        # Reuse the original prompt plus additional scraped content.
        prompt_text_extended = (
            f"I previously asked: Based on the following snippet and URL, determine if the website likely represents a nonprofit organization. "
            f"Answer only 'Yes' or 'No'.\n\nSnippet: {text}\n\nURL: {url}\n\n"
            f"I have also scraped additional context from the website: {additional_text}\n\n"
            f"Please answer with only 'Yes' or 'No'."
        )
        response = openai.chat.completions.create(
            model='anthropic.claude-3.5-sonnet.v2',
            messages=[{'role': 'user', 'content': prompt_text_extended}],
        )
    answer = response.choices[0].message.content.strip().lower()
    return answer.startswith("yes")

async def retrieve_nonprofit_candidates(prompt: str, max_candidates: int = 3, max_results: int = 100) -> list:
    """
    Use SERP API (with pagination) to search for the prompt.
    Only return candidate results that have '.org' in their URL and for which an AI (via is_nonprofit)
    judges as likely a nonprofit.
    Stop when either max_candidates have been collected or when max_results have been processed.
    """
    candidates = []
    total_processed = 0
    start = 0

    while total_processed < max_results and len(candidates) < max_candidates:
        url = f"https://serpapi.com/search.json?q={prompt}&engine=google&api_key={SERP_API_KEY}&start={start}"
        async with httpx.AsyncClient() as client:
            response = await client.get(url)
            data = response.json()
        organic_results = data.get("organic_results", [])
        if not organic_results:
            break

        for item in organic_results:
            if total_processed >= max_results:
                break
            total_processed += 1
            link = item.get("link", "")
            # Only consider URLs with .org
            if ".org" not in link:
                continue
            snippet = item.get("snippet", "")
            if await is_nonprofit(snippet, link):
                candidates.append({
                    "title": item.get("title", ""),
                    "link": link,
                    "snippet": snippet
                })
                if len(candidates) >= max_candidates:
                    break
        start += len(organic_results)
    return candidates

async def summarize_with_ai(text: str, url: str) -> str:
    """
    Generate a summary of the snippet with context from the given URL.
    """
    response = openai.chat.completions.create(
        model='anthropic.claude-3.5-sonnet.v2',
        messages=[
            {
                'role': 'user',
                'content': f"Summarize this snippet about a non-profit:\n\n{text}\n\nInclude the context from this website: {url}"
            },
        ],
    )
    summary = response.choices[0].message.content.strip()
    return summary

async def generate_nonprofit_summaries(candidates: list) -> list:
    """
    For each candidate result, generate a summary.
    """
    summaries = []
    for candidate in candidates:
        snippet_summary = await summarize_with_ai(candidate["snippet"], candidate["link"])
        summaries.append({
            "title": candidate["title"],
            "link": candidate["link"],
            "summary": snippet_summary
        })
    return summaries

async def full_search_summarize(prompt: str) -> list:
    """
    Full RAG pipeline: First retrieve candidate nonprofit websites,
    then generate summaries for each candidate.
    """
    candidates = await retrieve_nonprofit_candidates(prompt)
    return await generate_nonprofit_summaries(candidates)

# Example usage:
user_prompt_ex = "financial aid educational non profit"

async def generate_table_from_prompt(prompt: str):
    # This function composes the entire pipeline: retrieving and summarizing.
    search_results = await full_search_summarize(prompt)
    # Prepare data for display
    data = [{"Link": result["link"], "Summary": result["summary"]} for result in search_results]
    df = pd.DataFrame(data)
    print(df.to_string(index=False))

# Run the async function (assuming an async-capable environment)
#await generate_table_from_prompt(user_prompt_ex)

msg = await fetch_full_page("https://believeinstudents.org/")

''